# 목차

##### 1. 노드 실습
##### 2. 프로젝트

# 노드 실습

In [1]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [2]:
# 1차 필터링. 화자가 표시된 문장과 공백인 문장을 처리해줌

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [3]:
# 토큰화 (띄어쓰기를 기준으로)
# 3가지 문제 케이스
# 1. 문장 부호 2. 대소문자 3. 특수문자

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))


<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [5]:
# 우리가 처음 언어를 배울때 모국어로 된 표현으로 공부를 하듯, 컴퓨터도 숫자로 배우게 된다.
# 이 과정에서 텐서플로우를 사용. 이 패키지는 정제된 데이터를 토큰화 하고 단어사전을 만들어 주며, 데이터를
# 숫자로 변환까지 해줌. 이 과정을 벡터화 라고 하고 숫자로 변환된 데이터를 텐서라고 한다.

# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f2169686090>


In [6]:
# 텐서 데이터는 모두 정수로 이루어져 있습니다. 이 숫자는 다름 아니라, tokenizer에 구축된 단어 사전의 인덱스입니다.
# 단어 사전이 어떻게 구축되었는지 아래와 같이 확인해 봅시다.

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [7]:
# 텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워 넣은 것입니다. 
# 사전에는 없지만 0은 바로 패딩 문자 <pad>가 될 것입니다.

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [8]:
# corpus 내의 첫 번째 문장에 대해 생성된 소스와 타겟 문장을 확인해 보았습니다. 
# 예상대로 소스는 2(<start>)에서 시작해서 3(<end>)으로 끝난 후 0(<pad>)로 채워져 있습니다. 
# 하지만 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한 칸 시프트 한 형태를 가지고 있습니다

# 마지막으로 우리는 데이터셋 객체를 생성할 것입니다. 
# 그동안 우리는 model.fit(x_train, y_train, ...) 형태로 Numpy Array 데이터셋을 생성하여 model에 제공하는 형태의 학습을 많이 진행해 왔습니다. 
# 그러나 텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 tf.data.Dataset객체를 생성하는 방법을 흔히 사용합니다. 
# tf.data.Dataset객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의 기능을 제공하므로 꼭 사용법을 알아 두시기를 권합니다. 
# 우리는 이미 데이터셋을 텐서 형태로 생성해 두었으므로, tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성할 것입니다.

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [9]:
# 학습시키기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

Embedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔 줍니다. 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용됩니다.

위 코드에서 embedding_size 는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기입니다. 만약 그 크기가 2라면 예를 들어

- 차갑다: [0.0, 1.0]
- 뜨겁다: [1.0, 0.0]
- 미지근하다: [0.5, 0.5]

값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기할 수 있습니다. 이번 실습에서는 256이 적당해 보이네요.

STM 레이어의 hidden state 의 차원수인 hidden_size 도 같은 맥락입니다. hidden_size 는 모델에 얼마나 많은 일꾼을 둘 것인가? 로 이해해도 크게 엇나가지 않습니다. 그 일꾼들은 모두 같은 데이터를 보고 각자의 생각을 가지는데, 역시 충분한 데이터가 주어지면 올바른 결정을 내리겠지만 그렇지 않으면 배가 산으로 갈 뿐 입니다

In [12]:
# 아직 모델을 빌드 하지 않았다. 이럴때 모델에 데이터를 조금 넣어보면 좋다.

# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-4.24588827e-04,  2.55890220e-04,  1.57685732e-04, ...,
          3.98655538e-04,  1.09762273e-04,  1.10298795e-04],
        [-4.29135747e-04,  4.31284046e-04,  1.42029283e-04, ...,
          7.29580468e-04,  3.09079682e-04,  2.49494420e-04],
        [-4.11398709e-04,  3.26509587e-04,  6.33867894e-05, ...,
          8.25205410e-04,  4.08633263e-04,  2.50226527e-04],
        ...,
        [ 1.12994062e-03,  1.02787639e-03,  7.53116386e-04, ...,
         -1.78548321e-03, -1.51731481e-03,  4.61191608e-04],
        [ 1.20162778e-03,  1.46226631e-03,  1.01114088e-03, ...,
         -1.93355081e-03, -1.80225563e-03,  2.07585745e-05],
        [ 1.25800772e-03,  1.89239380e-03,  1.23049412e-03, ...,
         -2.07765540e-03, -2.08243821e-03, -4.02214617e-04]],

       [[-4.24588827e-04,  2.55890220e-04,  1.57685732e-04, ...,
          3.98655538e-04,  1.09762273e-04,  1.10298795e-04],
        [-2.19430003e-04,  6.88256056e-04,  7.

모델의 최종 출력 텐서 shape를 유심히 보면 shape=(256, 20, 7001)임을 알 수 있습니다. 7001은 Dense 레이어의 출력 차원수입니다. 7001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야 하기 때문입니다.
256은 이전 스텝에서 지정한 배치 사이즈입니다. dataset.take(1)를 통해서 1개의 배치, 즉 256개의 문장 데이터를 가져온 것입니다.

그렇다면 20은 무엇을 의미할까요? 비밀은 바로 tf.keras.layers.LSTM(hidden_size, return_sequences=True)로 호출한 LSTM 레이어에서 return_sequences=True이라고 지정한 부분에 있습니다. 즉, LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미입니다. 만약 return_sequences=False였다면 LSTM 레이어는 1개의 벡터만 출력했을 것입니다.

그런데 문제는, 우리의 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모른다는 점입니다. 모델을 만들면서 알려준 적도 없습니다. 그럼 20은 언제 알게된 것일까요? 네, 그렇습니다. 데이터를 입력받으면서 비로소 알게 된 것입니다. 우리 데이터셋의 max_len이 20으로 맞춰져 있었던 것입니다.

In [11]:
# 이제 모델이 학습할 준비가 완료되었습니다. 아래 코드를 실행해 모델을 학습시켜보세요!

# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 39s 368ms/step - loss: 4.3828
Epoch 2/30
93/93 [==============================] - 34s 370ms/step - loss: 2.8137
Epoch 3/30
93/93 [==============================] - 35s 373ms/step - loss: 2.7145
Epoch 4/30
93/93 [==============================] - 35s 375ms/step - loss: 2.6052
Epoch 5/30
93/93 [==============================] - 35s 379ms/step - loss: 2.5316
Epoch 6/30
93/93 [==============================] - 35s 380ms/step - loss: 2.4877
Epoch 7/30
93/93 [==============================] - 35s 381ms/step - loss: 2.4250
Epoch 8/30
93/93 [==============================] - 36s 382ms/step - loss: 2.3780
Epoch 9/30
93/93 [==============================] - 35s 381ms/step - loss: 2.3096
Epoch 10/30
93/93 [==============================] - 35s 381ms/step - loss: 2.2609
Epoch 11/30
93/93 [==============================] - 36s 383ms/step - loss: 2.2083
Epoch 12/30
93/93 [==============================] - 36s 382ms/step - loss: 2.1567
Epoch 13/30
9

In [12]:
# 모델 평가

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

학습 단계에서 우리는 이런 while 문이 필요 없었습니다. 소스 문장과 타겟 문장이 있었고, 우리는 소스 문장을 모델에 입력해서 나온 결과를 타겟 문장과 직접 비교하면 그만이었습니다.
그러나 텍스트를 실제로 생성해야 하는 시점에서, 우리에게는 2가지가 없습니다. 하나는 타겟 문장입니다. 또 하나는 무엇이냐 하면, 소스 문장입니다. 생각해 보면 우리는 텍스트 생성 태스크를 위해 테스트 데이터셋을 따로 생성한 적이 없습니다.

generate_text() 함수에서 init_sentence를 인자로 받고는 있습니다. 이렇게 받은 인자를 일단 텐서로 만들고 있습니다. 디폴트로는 `<start>` 단어 하나만 받는군요.
    
    
while의 첫 번째 루프에서 test_tensor에 `<start>` 하나만 들어갔다고 합시다. 우리의 모델이 출력으로 7001개의 단어 중 A를 골랐다고 합시다.
while의 두 번째 루프에서 test_tensor에는 `<start>` A가 들어갑니다. 그래서 우리의 모델이 그다음 B를 골랐다고 합시다.
while의 세 번째 루프에서 test_tensor에는 `<start>` A B가 들어갑니다. 그래서..... (이하 후략)

In [13]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s not prepared for death . <end> '

# 프로젝트: 멋진 작사가 만들기

In [14]:
# 데이터 읽어오기

import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free']


In [15]:
# 데이터 정제

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue 
    if sentence[-1] == "]": continue  # [verse] 부분 삭제
    if sentence[-1] == ":": continue

    if idx > 9: break
        
    print(sentence)
    
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))


They come from everywhere
A longing to be free
They come to join us here
From sea to shining sea And they all have a dream
As people always will
To be safe and warm
In that shining city on the hill Some wanna slam the door
Instead of opening the gate
Aw, let's turn this thing around
<start> this is sample sentence . <end>


In [16]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == "]": continue

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

corpus[:10]

['<start> they come from everywhere <end>',
 '<start> a longing to be free <end>',
 '<start> they come to join us here <end>',
 '<start> from sea to shining sea and they all have a dream <end>',
 '<start> as people always will <end>',
 '<start> to be safe and warm <end>',
 '<start> in that shining city on the hill some wanna slam the door <end>',
 '<start> instead of opening the gate <end>',
 '<start> aw , let s turn this thing around <end>',
 '<start> it s up to me and you <end>']

In [17]:
# 벡터화


def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 토큰의 개수를 15개로 지정
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)

    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   45   66 ...    0    0    0]
 [   2    9 3375 ...    0    0    0]
 [   2   45   66 ...    0    0    0]
 ...
 [   2  563   21 ...    0    0    0]
 [   2  120   34 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f204cac6650>


In [18]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [19]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  45  66  74 799   3   0   0   0   0   0   0   0   0]
[ 45  66  74 799   3   0   0   0   0   0   0   0   0   0]


In [20]:
# 평가 데이터셋 분리

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=1)

In [21]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


Source Train: (139790, 14)
Target Train: (139790, 14)


In [22]:
# 인공지능 만들기

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [23]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [24]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.0140727e-04,  2.0616390e-04, -4.2554113e-04, ...,
         -1.5800384e-04,  2.1031841e-04, -2.3018006e-04],
        [ 2.1496012e-04,  3.2461269e-04, -6.9152931e-04, ...,
         -1.4715282e-04,  2.6054820e-04,  8.2834158e-05],
        [ 3.6073948e-04,  8.8138768e-04, -9.1926730e-04, ...,
         -5.7281206e-05,  5.6868704e-04, -4.9744444e-06],
        ...,
        [ 9.3532767e-04,  3.3292603e-03, -1.3484472e-03, ...,
         -6.7793665e-04,  7.4223639e-04, -3.8729122e-04],
        [ 7.2741375e-04,  3.0121945e-03, -1.0211664e-03, ...,
         -4.7501744e-04,  7.0931012e-04, -4.0866400e-04],
        [ 2.6291996e-04,  2.5508192e-03, -7.5002154e-04, ...,
         -5.6640233e-04,  1.0886755e-03, -3.9751059e-04]],

       [[ 3.4233587e-04, -1.3250958e-04,  1.7199527e-04, ...,
          9.1632945e-05,  1.5460582e-04, -1.9369171e-04],
        [ 5.2243553e-04, -5.4761732e-04,  2.3239070e-04, ...,
          1.7637172e-04, 

In [25]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  6144512   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  20979712  
_________________________________________________________________
lstm_3 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_1 (Dense)              multiple                  24590049  
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


In [26]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

# validation loss
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), verbose=1)

Epoch 1/10
682/682 [==============================] - 714s 1s/step - loss: 3.9042 - val_loss: 2.8849
Epoch 2/10
682/682 [==============================] - 713s 1s/step - loss: 2.8163 - val_loss: 2.4102
Epoch 3/10
682/682 [==============================] - 713s 1s/step - loss: 2.3799 - val_loss: 2.0081
Epoch 4/10
682/682 [==============================] - 712s 1s/step - loss: 1.9943 - val_loss: 1.6638
Epoch 5/10
682/682 [==============================] - 713s 1s/step - loss: 1.6663 - val_loss: 1.3935
Epoch 6/10
682/682 [==============================] - 711s 1s/step - loss: 1.3999 - val_loss: 1.1940
Epoch 7/10
682/682 [==============================] - 710s 1s/step - loss: 1.2050 - val_loss: 1.0525
Epoch 8/10
682/682 [==============================] - 710s 1s/step - loss: 1.0713 - val_loss: 0.9683
Epoch 9/10
682/682 [==============================] - 711s 1s/step - loss: 0.9888 - val_loss: 0.9208
Epoch 10/10
682/682 [==============================] - 710s 1s/step - loss: 0.9461 - val_lo

In [27]:
# 모델 평가

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):

    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:

        predict = model(test_tensor) 

        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 

        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [28]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love the way you shake that thing girl <end> '

# conclusion

다행히 모델이 정상적으로 작동하고 결과물도 내 기준에서는 사람이 만든 가사처럼 느껴졌다. 물론 모든 코드를 다 이해한것도 아니고 설령 그런 부분이 있었더라도 다른 프로젝트에서 어떻게 이용할지는 아직 감이 잡히지 않는다. 그건 미래의 내가 하겠지.... 이번 노드를 할 때 조금 힘든 부분은 loss를 줄이는 부분이었다. 다만, 이 부분은 embedding size와 hidden size를 두배로 지정하니 loss값이 줄어들었다. 확실한건 아니지만, 데이터를 더 많이 넣었을때 정확도가 조금 올라가는 원리와 비슷한게 아닐까 싶다. 이번 노드를 하면서 느낀 점은 아직도 내가 너무 부족하다는 것이다. 늘 하는 이야기지만 열심히 하는 수 밖에는 없을거 같다!